# Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

El objetivo es desarrollar un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usaremos el data set para comprobar la exactitud.

## Importar librerias

In [44]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## Importar base de datos para análisis

In [45]:
# # Importar dataset
df = pd.read_csv('/datasets/users_behavior.csv')


## Explorar dataset

In [46]:
# Explorar dataset
print(df.shape)
print()
print("*****************************************************************")
print(df.head())
print()
print("*****************************************************************")
print(df.info())
print()
print("*****************************************************************")
print(df.describe())



(3214, 5)

*****************************************************************
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

*****************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

*****************************************************************
             calls      minutes     messages       mb_used     is

Los tipo de datos estan asignados corretamente. Vemos que el dataset incluye las llamadas, minutos, mensajes y megas usados en cada plan. Se tiene una columna que identifica si el plan es ultra con un 1, y si dice 0, se refiere a que es el plan smart.

## Segmentar datos

Vamos a segmentar los datos en un conjunto de entrenamiento, uno de validación y otro de prueba. Separaré la base de datos en una proporción de 60%, 20% y 20% respectivamente

In [47]:
# Separar el dataset en entrenamiento, validación y prueba
df_train, df_temp = train_test_split(df, test_size=0.4, random_state=12345)
df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=12345)

# Declarar variables para las caraceteristicas y para la caracteristica objetivo
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## Investigar exactitud de varios modelos

Probaré tres modelos para analizar mi informacón: Decision Tree Classifier, Random Forest Classifer y Logist Regression.



In [48]:
# Modelo DecisionTreeClassifier
best_score_dtc = 0
best_depth = 0
for depth in range(1,29):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dtc.fit(features_train,target_train)
    predictions_valid = model_dtc.predict(features_valid)
    score = accuracy_score(target_valid,predictions_valid)
    if score > best_score_dtc:
        best_score_dtc = score
        best_depth = depth
print("DecisionTreeClassifier")
print("El mejor max_depth =", best_depth, "con una exactitud de:", best_score_dtc)
print()

# Modelo RandomForestClassifier
best_score_rfc = 0
best_est = 0
for est in range(1, 15): # selecciona el rango del hiperparámetro
    model_rfc = RandomForestClassifier(random_state=12345, n_estimators= est) # configura el número de árboles
    model_rfc.fit(features_train, target_train) # entrena el modelo en el conjunto de entrenamiento
    score = model_rfc.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación
    if score > best_score_rfc:
        best_score_rfc = score # guarda la mejor puntuación de accuracy en el conjunto de validación
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud
print("RandomForestClassifier")
print("El mejor n_estimators =", best_est, "con una exactitud de:", best_score_rfc)
print()

# Modelo LogisticRegression
model_lr = LogisticRegression(random_state=12345, solver='liblinear') # inicializa el constructor de regresión logística con los parámetros random_state=54321 y solver='liblinear'
model_lr.fit(features_train,target_train) # entrena el modelo en el conjunto de entrenamiento
score_train = model_lr.score(features_train,target_train) # calcula la puntuación de accuracy en el conjunto de entrenamiento
score_valid = model_lr.score(features_valid,target_valid) # calcula la puntuación de accuracy en el conjunto de validación
print("LogisticRegression")
print("Exactitud del set de entrenamiento:", score_train)
print("Exactitud del set de validación:", score_valid)

DecisionTreeClassifier
El mejor max_depth = 3 con una exactitud de: 0.7853810264385692

RandomForestClassifier
El mejor n_estimators = 12 con una exactitud de: 0.7869362363919129

LogisticRegression
Exactitud del set de entrenamiento: 0.7505186721991701
Exactitud del set de validación: 0.7589424572317263


En los modelos Decision Tree Classifer y Random Forest Classifier modifique los parametros max_depth y n_estimators respectivamente, y e itere sobre un rango para encontrar cual era el mejor hiperparametro. El tiempo de respuesta fue realmente rápido en ambos casos. 

En el caso del modelo Logistic Regression utilice el parametro liblinear, que era el mejor para la cantidad de datos que tenemos por analizar.

## Escoger el mejor modelo usando el conjunto de prueba



In [39]:
# Escoger el mejor modelos y evaluarlo con el set de prueba
best_model = None
best_model_name = ""
if max(best_score_dtc, best_score_rfc, score_valid) == best_score_dtc:
    best_model = DecisionTreeClassifier(random_state=12345, max_depth=best_depth)
    best_model_name = "DecisionTreeClassifier"
elif max(best_score_dtc, best_score_rfc, score_valid) == best_score_rfc:
    best_model = RandomForestClassifier(random_state=12345, n_estimators=best_est)
    best_model_name = "RandomForestClassifier"
else:
    best_model = LogisticRegression(random_state=12345, solver='liblinear')
    best_model_name = "LogisticRegression"

best_model.fit(features_train, target_train)
predictions_test = best_model.predict(features_test)
score_test = accuracy_score(target_test, predictions_test)


print("\nEl mejor modelo es", best_model_name)
print("La exactitud del modelo", best_model_name, "en el set de prueba es :", score_test)


El mejor modelo es  RandomForestClassifier
El rendimiento del modelo RandomForestClassifier en el set de prueba es : 0.7869362363919129


Finalmente, utilice el set de prueba y lo probé en los tres modelos para saber cual era el que tenia mayor exactitud, y el resultado fue que Random Forest Classifier era el mejor.